# Question
You receive a distress call from the middle of the North Atlantic Ocean. The person on
the line gave you a coordinates of lat: 32.610982, long: -38.706256 and asked for the nearest
port with provisions, water, fuel_oil and diesel. Your answer should include the columns
country, port_name, port_latitude and port_longitude only.

In [ ]:
import psycopg2
import os 

## Connect to the PostgreSQL database

In [ ]:
def get_pg_connection(host, database, user, password):
    pg_conn = psycopg2.connect(host = host, database = database, user = user, password = password)
    return pg_conn
pg_conn = get_pg_connection('localhost', 'Capstone', os.environ['user'], os.environ['password'])
pg_cur = pg_conn.cursor()

## SQL Query 

In [ ]:
lat= 32.610982
long= -38.706256

sql_query = f""" 
    CREATE TABLE nearest_port_with_provvisions AS
    SELECT "Wpi_country_code" AS country,"Main_port_name" AS port_name, "Longitude_degrees" AS port_longitude, "Latitude_degrees" AS port_latitude 
    FROM wpi_data 
    WHERE "Supplies_provisions" = 'Y'AND "Supplies_water" = 'Y'AND "Supplies_fuel_oil" = 'Y'AND "Supplies_diesel_oil" = 'Y'
    ORDER BY (6371000 * ACOS(
        SIN(RADIANS({lat})) * SIN(RADIANS("Latitude_degrees")) +
        COS(RADIANS({lat})) * COS(RADIANS("Latitude_degrees")) *
        COS(RADIANS("Longitude_degrees") - RADIANS({long}))
    ))  
    LIMIT 1
"""
pg_cur.execute(sql_query)

pg_conn.commit()

## Close the cursor and connection

In [ ]:
pg_cur.close()
pg_conn.close()